# Sklearn: train_test_split

머신러닝 모델을 학습하고, 그 결과를 검증하기 위해서는 원래의 데이터를 Training, Validation, Testing의 용도로 나누어 다뤄야 한다. 
Training에 사용한 데이터를 검증용으로 사용하면 답을 아는 상태에서 공부를 하는 것이기에, 제대로 된 검증이 이루어지지 않는다.

교차 검증을 하려면 두 종류의 데이터 집합 필요
 - training data set
 - test data set
 
-> 가지고 있는 데이터 집합을 학습용과 검증용으로 나누어주는 함수: skelarn의 train_test_split

In [66]:
import numpy as np

X = [[0,1],[2,3],[4,5],[6,7],[8,9]]
Y = [0,1,2,3,4]

## 1. Parameter 


train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)

- arrays: 분할시킨 데이터를 입력 (Numpy array, Pandas dataframe..)
- test_size: 테스트 데이터셋의 비율이나 객수 (default = 0.25)
- train_size: 학습 데이터셋의 비율이나 갯수 (1 - test_size)
- random_state: 데이터 분할시 셔플이 이루어지는데, 이를 위한 시드값 (int를 입력하면 숫자를 random하게 생성할 때 사용되는 seed 숫자 사용, None을 입력하면 np.random에서 제공하는 random number generator 사용)
- shuffle: 셔플 여부 결정 (default = True)
- stratify: 지정한 data의 비율 유지, label set인 Y가 25%의 0과 75%의 1로 이루어진 binary set일 때, stratify = Y로 설정하면 나누어진 데이터셋들도 0과 1을 각각 25%,  75%로 유지한 채 분할된다.

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, 
                                   test_size = 0.2, 
                                   train_size = 0.8, 
                                   random_state = 123, 
                                   shuffle = True)

In [70]:
print(X_train)
print(Y_train)

[[6, 7], [8, 9], [0, 1], [4, 5]]
[2, 0, 3]


In [73]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.33,
                                                    random_state=42, 
                                                    shuffle = False)

In [74]:
print(X_train)
print(Y_train)

[[0, 1], [2, 3], [4, 5]]
[0, 1, 2]


## 2. Return: list 형태

- 클래스 값을 포함하여 하나의 데이터로 받는 경우: X_train, X_test 
      arrays에 레이블 없이 데이터만 넣었을 경우의 반환
- 클래서를 개별의 배열로 받는 경우: X_train, X_test, Y_train, Y_test 
      arrays에 데이터와 레이블을 둘 다 넣었을 경우의 반환이며, 데이터와 레이블의 순서쌍은 유지된다.

# 흔히 하는 실수 정리

### 1. 순서 오류: X_test, X_train, y_test, y_train

### 2. train_size, test_size 혼동 (하나만 사용할 것)
  
### 3. 정규화가 필요한 경우 (StandardScaler 함수 사용)  
    ex: 단위가 다른 키(cm)와 무게(kg) 
    
### 4. Shuffle = False 설정
    ex: 남성 50, 여성 50인 경우에 데이터를 80-20으로 나누면, 50의 남성과 30의 여성으로 구성된 데이터 세트로 학습, 20의 여성만 포함된 데이터 세트로 테스트해야 함
  
### 5. stratify 매개 변수
    array와 같은 object로 설정하지 않으면, 목표 그룹의 분포를 나타내지 못할 수 있다.
   
### 6. random_state 매개 변수 설정 잊어버리기

# 다른 무작위 램덤 추출 방법

## 1. 사용자 정의 함수

In [1]:
import numpy as np
import pandas as pd


# setting seed number for reproducibility

np.random.seed(123)


# 데이터프레임으로 변환

df = pd.DataFrame({'grp': ['grp_1']*10 + ['grp_2']*10, 

                          'col_1': np.random.randint(20, size=20), 
                          'col_2': np.random.randint(20, size=20)})

In [2]:
df.head()

,grp,col_1,col_2
0,grp_1,13,16
1,grp_1,2,4
2,grp_1,2,17
3,grp_1,6,3
4,grp_1,17,2


In [3]:
def sampling_func(data, sample_pct):
    np.random.seed(123)
    N = len(data)
    sample_n = int(len(data)*sample_pct) # integer
    sample = data.take(np.random.permutation(N)[:sample_n])

    return sample

In [5]:
# 80%만큼 무작위 표본 추출

sample_set = df.groupby('grp', group_keys = False).apply(sampling_func, sample_pct=0.8)

sample_set.sort_index()

,grp,col_1,col_2
0,grp_1,13,16
1,grp_1,2,4
3,grp_1,6,3
4,grp_1,17,2
5,grp_1,19,7
6,grp_1,10,2
7,grp_1,1,15
8,grp_1,0,16
10,grp_2,15,9
11,grp_2,9,3


In [6]:
# 나머지 20%로 test_set 별도로 만들기

test_set = df.drop(df.index[sample_set.index]) ; test_set

,grp,col_1,col_2
2,grp_1,2,17
9,grp_1,17,7
12,grp_2,0,6
19,grp_2,0,10


## 2. pandas의 sample method

In [22]:
np.random.seed(123)

df = pd.DataFrame({'col_1': np.random.randint(50, size=100), 
                   'col_2': np.random.randint(50, size=100)})

In [23]:
df.head()

,col_1,col_2
0,45,6
1,2,9
2,28,23
3,34,14
4,38,38


In [24]:
# 테스트 셋 분리

sample_1 = df.sample(frac = 0.70, random_state = 2019)
len(sample_1)

70

In [25]:
# 검증 셋 분리

sample_2 = df.drop(sample_1.index)
len(sample_2)

30